In [1]:
%load_ext autoreload
%autoreload 2
import time

In [2]:
import json
import design as d
import names as n

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)

from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys
close_ansys()

In [3]:
# To store the data in the "out" folder
import os

os.makedirs(os.path.dirname("out/"), exist_ok=True)

## Design assembly

In [4]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um", material='silicon')
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

n.add_design_variables_to_design(design, initial_design_variables)

In [5]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, group=n.NBR_1)
    d.add_transmon_plus_resonator(design, group=n.NBR_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, group=n.NBR_1)
    d.add_chargeline(design, group=n.NBR_2)

    gui.rebuild()
    gui.autoscale()


render_qiskit_metal_design(design, gui)

position: [ 0.       -2.721745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
position: [0.       4.461745] : direction:[-0. -1.]. position: [0.       3.068255] : direction:[0. 1.].
position: [ 0.       -4.461745] : direction:[0. 1.]. position: [ 0.       -3.078255] : direction:[ 0. -1.].
position: [-4.37 -2.  ] : direction:[ 1. -0.]. position: [-4.2 -1.8] : direction:[-1.  0.].
position: [-4.37  2.  ] : direction:[ 1. -0.]. position: [-4.2  1.8] : direction:[-1.  0.].
position: [ 0.       -2.721745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
position: [0.       4.461745] : direction:[-0. -1.]. position: [0.       3.068255] : direction:[0. 1.].
position: [ 0.       -4.461745] : direction:[0. 1.]. position: [ 0.       -3.078255] : direction:[ 0. -1.].
position: [-4.37 -2.  ] : direction:[ 1. -0.]. position: [-4.2 -1.8] : direction:[-1.  0.].
position: [-4.37  2.  ] : direction:[ 1. -0.]. position: [-4.2  1.8] : direction:[-1.  0

## Two qubit-resonator system

In [6]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

In [7]:
MINI_STUDY = ms.get_mini_study_2qb_resonator_coupler()
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[n.NBR_1, n.NBR_2],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
    opt_target_coupler_freq=True,
)

In [8]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_TWO_QB,
)

group_runs = 15
group_passes = 7
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-03-11 16:03:12]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 04:03PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:03PM [load_ansys_project]: 	Opened Ansys App
INFO 04:03PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 04:03PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/harshva/Documents/Ansoft/
	Project:   Project218
INFO 04:03PM [connect_design]: No active design found (or error getting active design).
INFO 04:03PM [connect]: 	 Connected to project "Project218". No design detected
04:03PM 32s WARNING [activate_ansys_design]: The design_name=get_mini_study_2qb_resonator_coupler was not in active project.

Initializing new desktop!
PyAEDT INFO: StdOut is enabled
PyAEDT INFO: Log on file is enabled
PyAEDT INFO: Log on Desktop Message Manager is enabled
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\AnsysEM21.2\Win64.
PyAEDT INFO: Launching AEDT with COM plugin using PythonNET.
PyAEDT INFO: AEDT 2021.2 Started with process ID 34460.
PyAEDT INFO: pyaedt v0.6.94
PyAEDT INFO: Python version 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:07:43) [MSC v.1942 64 bit (AMD64)]
PyAEDT INFO: AEDT 2021.2.0 Build Date 2021-06-02 22:21:55
PyAEDT INFO: No project is defined. Project Project218 exists and has been read.
PyAEDT INFO: Active Design set to get_mini_study_2qb_resonator_coupler
PyAEDT INFO: Aedt Objects correctly read
position: [ 0.       -2.721745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
pos

[INFO|2025-03-11 16:03:48]: No fine mesh map was found for name_qubit_1
[INFO|2025-03-11 16:03:48]: No fine mesh map was found for name_resonator_1
[INFO|2025-03-11 16:03:48]: No fine mesh map was found for name_tee1
[INFO|2025-03-11 16:03:48]: No fine mesh map was found for name_qubit_2
[INFO|2025-03-11 16:03:48]: No fine mesh map was found for name_resonator_2
[INFO|2025-03-11 16:03:48]: No fine mesh map was found for name_tee2
[INFO|2025-03-11 16:03:48]: No fine mesh map was found for name_coupler_1to2
INFO 04:03PM [analyze]: Analyzing setup Setup


Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1

        energy_elec_all       = 8.71077993016417e-24
        energy_elec_substrate = 7.91018001743196e-24
        EPR of substrate = 90.8%

        energy_mag    = 9.9137945026036e-26
        energy_mag % of energy_elec_all  = 1.1%
        

Variation 0  [1/1]

  Mode 0 at 5.63 GHz   [1/5]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               98.9%  4.355e-24 4.957e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0

WARNING 04:07PM [__init__]: <p>Error: <class 'IndexError'></p>


...

WARNING 05:06PM [__init__]: <p>Error: <class 'IndexError'></p>
ERROR 05:06PM [_get_participation_normalized]: WARNING: U_tot_cap-U_tot_ind / mean = 200.0% is > 15%.                     
Is the simulation converged? Proceed with caution



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project218\get_mini_study_2qb_resonator_coupler\2025-03-11 17-06-10.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization


ERROR 05:06PM [_get_participation_normalized]: WARNING: U_tot_cap-U_tot_ind / mean = 200.0% is > 15%.                     
Is the simulation converged? Proceed with caution


Pm_norm=
modes
0         1.048127
1    410582.845536
2         2.693179
3         2.180377
4    531953.857324
dtype: float64

Pm_norm idx =
   jj_name_qubit_1  jj_name_qubit_2
0            False            False
1            False            False
2            False            False
3            False            False
4            False            False
*** P (participation matrix, not normlz.)
   jj_name_qubit_1  jj_name_qubit_2
0     4.431479e-05     4.512712e-11
1     3.303436e-17     2.434930e-06
2     5.780719e-07     1.819769e-09
3     3.445578e-11     2.188683e-06
4     2.713045e-11     1.879469e-06

*** S (sign-bit matrix)
   s_jj_name_qubit_1  s_jj_name_qubit_2
0                  1                 -1
1                  1                 -1
2                 -1                  1
3                  1                 -1
4                  1                  1
*** P (participation matrix, normalized.)
   4.4e-05  4.5e-11
   3.3e-17  2.4e-06
   5.8e-07  1.8e-09
   3.4e-11  2.2e-06

[INFO|2025-03-11 17:06:48]: freq_ND_results
{
    "0": {
        "0": 4993.588327820032,
        "1": 5122.1622579069945,
        "2": 6230.874181479862,
        "3": 7010.2199056494655,
        "4": 7930.113439455209
    }
}
05:06PM 49s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: QEventDispatcherWin32::wakeUp: Failed to post a message (Not enough quota is available to process this command.)



In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

In [ ]:
design_analysis.get_eigenmode_results()

In [ ]:
design_analysis.overwrite_parameters()

In [ ]:
close_ansys()